# Case Study 1 : Collecting Data from Twitter

Due Date: September 22, **before the beginning of class at 6:00pm**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:**

    Matthew Beaulieu
    
    Yousef Fadila
    
    Meng Li
    
    Monica Tlachac
    
---

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [ ]:
import pandas as pd
import matplotlib
import warnings
import json
import numpy as np
import wikiwords
from tweepy import Stream
from tweepy.streaming import StreamListener
import pymongo
import tweepy
import re

warnings.simplefilter(action = "ignore", category = FutureWarning)
%matplotlib inline
#---------------------------------------------
# Define a Function to Login Twitter API
#
# XXX  - DON'T COMMIT TRUE CREDINTIALS TO GITHUB
CONSUMER_KEY = 'k01qrrsDSOHdjUObbt1pWjhld'
CONSUMER_SECRET ='FTBSa4UU5Nxaw00hNPJQvoAf60HinJM87GfQpuFM0YsVKDdoJD'
OAUTH_TOKEN = '4229853605-mSgVKfVSZ9BSN7GbPgvUIPOT1Xl6F3nA2ynKMvy'
OAUTH_TOKEN_SECRET = 'jyHtNVKGmqNw4HyBLMYG5i9BT3junKGVFnY6CE2tjJRFW'

# another set of auth to use when limit is hit.
#CONSUMER_KEY = '7n45AnattFU5sb3MpbiwIoNYF'
#CONSUMER_SECRET ='VtnEmlR99dG3hQ8FP8xb1TnomTAjSGkQYDJVhGa28IDPDEA2Cf'
#OAUTH_TOKEN = '2497861306-W7d3EXegd4AY6jYt9ZaEz5rq5ZtguAAtp6KbseS'
#OAUTH_TOKEN_SECRET = 'uv71S6pJwPibOFLA8WetoxWRb0nfqcQPK8f3ts30nEfQT'

# mango db auth.
MANGODB_SERVER = 'ds033046.mlab.com'
MANGODB_PORT = 33046
MANGODB_USER = 'ds501u'
MANGODB_PASS = 'ds501p'
MANGODB_NAME = 'ds501case1'
MANGODB_COLLECTION = 'tweets'
GEOBOX_USA = [-125,25.1,-60.5,49.1]

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

db_client = pymongo.MongoClient(MANGODB_SERVER, MANGODB_PORT)
db = db_client[MANGODB_NAME]
db.authenticate(MANGODB_USER, MANGODB_PASS)
#----------------------------------------------

In [17]:
class MyListener(StreamListener):
    def __init__(self, max_count = 100):
        self.count = 0
        self.max_count = max_count   
        self.collection = db[MANGODB_COLLECTION]

    def on_data(self, data):
        try:
            if self.count >= self.max_count:
                return False
            # the question require output to file. so output to file and save in mangoDB
            with open('data/hillary.json', 'a') as f:
                f.write(data)
            self.count = self.count + 1
            datajson = json.loads(data)
            self.collection.insert_one(datajson)
            return True
        except BaseException as e:
            print(e)
        return True
 
    def on_error(self, status):
        print(status)
        return True
    
api = tweepy.API(auth)
limit = 20000
twitter_stream = Stream(auth, MyListener(limit))
twitter_stream.filter(track=['@HillaryClinton'], languages=["en"], locations=GEOBOX_USA)
print("streaming done")


streaming done


### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

**Getting Text from Tweets** 

* Get the text column of the tweet df
* make it lower case
* remove http links and non alphanum characters

In [21]:
################# RE WRITE THIS - DF is not defined in this context. define it or loop over database/file. your coice
text = df.text.str.lower()
for i,t in enumerate(text):
    t = re.sub(r"(?:\@|https?\://)\S+", "", t)
    new = ''.join(e for e in t if (e.isalnum() or e==' '))
    text[i] = new
text = text.str.split(' ')

Make a dictionary with each unique word as the key, and the number of times it appears as the value

In [22]:
word_dict = {}
for r in range(text.shape[0]):
    lis = text.values[r]
    for word in lis:
        if word in word_dict.keys():
            word_dict[word] +=1
        else:
            word_dict[word] = 1

Add the key, value pairs to a dictionary (after removing rt, lingering https, and hillaryclinton)

In [23]:
vals = []
remove_list = ['', ' ', 'rt', 'http','https','hillaryclinton']
keys = [ e for e in word_dict.keys() if e not in remove_list]
for key in keys:
    vals.append(word_dict.get(key))
    
tweet_df = pd.DataFrame({'words':keys, 'vals':vals})
tweet_df = tweet_df.sort_values('vals', ascending=False)
tweet_df[0:30]

,vals,words
10186,4282,to
5108,4097,the
8577,2886,is
4799,2681,a
2245,2525,you
6185,2281,of
264,2036,for
9372,1883,in
5315,1666,and
7499,1575,on


Upon first anlysis, the 30 most used words in the tweets we pulled are pretty common, with special cases such as "hillary", and "trump".  

The next strategy was to normalize each tweet for how popular it is in the English language, to find a more interesting top word count 

In [24]:
# wikiwords is a cool library that tells how often a word appears in wikipedia articles
# and works to normalize for word popularity
tweet_df['freq'] = tweet_df['words'].apply(wikiwords.freq)
tweet_df = tweet_df[tweet_df['freq'] > 1.0e-6]
tweet_df['ratio'] = tweet_df['vals'] / tweet_df['freq']
tweet_df.sort_values('ratio', ascending=False)[0:30]

,vals,words,freq,ratio
6154,968,amp,0.000004,2.649444e+08
8935,391,hypocrisy,0.000002,2.041077e+08
4255,993,trump,0.000006,1.673466e+08
8735,915,hillary,0.000006,1.660476e+08
2046,236,unfit,0.000002,1.062605e+08
5565,104,yells,0.000001,1.025306e+08
6215,290,stunning,0.000004,8.027490e+07
4732,97,yrs,0.000001,7.345456e+07
10006,115,bodyguards,0.000002,7.057417e+07
3828,82,benghazi,0.000001,6.487133e+07


This list is much more interesting, with words like Benghazi, stunning, hypocrisy, and unfit, seeming like more specific to the subject being discussed

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.

As the tweets are already loaded into a Pandas DataFrame, this part should be pretty straightforward...

In [25]:
df.sort_values('retweet_count', ascending='False')[['retweet_count', 'text']][0:10]

,retweet_count,text
0,0,@BoyGeorge @HillaryClinton Don't be sorry for ...
9659,0,@HillaryClinton get off my feed
9660,0,RT @Delo_Taylor: After 8 yrs of neglect all @P...
9661,0,Please retweet twitter blocking comments https...
9662,0,@ClintonFdn @HillaryClinton \n\nhttps://t.co/y...
9663,0,RT @Bikers4Trump: After the 3 Terrorist Attack...
9664,0,@BarbaraAResEsq @Cuckerella let's not forget @...
9665,0,RT @FoxNews: .@JudgeJeanine to @HillaryClinton...
9666,0,RT @Someweirdid: @BrianAbelTV @HillaryClinton ...
9667,0,@HillaryClinton At some point you MUST address...


Unfortunately, these tweets were pulled from the twitter streaming API, meaning we saved the tweets as they appeared, and before they were retweeted or favorited by any of the user base.

To fix this, we pulled 1/10 of them again from the REST API (not the whole set due to rate limits), saved it to a JSON, loaded it to a df, and ran the command again

In [152]:
lst = []
with open('data/update_hillary.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        lst.append(tweet)
new_df = pd.DataFrame(lst)

In [163]:
#drop duplicates is used because on retweeted tweets, twitter stores
# the retweet count from the source tweet, for all of them, so there were several repeats
new_df.sort_values('retweet_count', ascending=False)[['user', 'retweet_count', 'text']][0:100].drop_duplicates('retweet_count')[0:10]

,user,retweet_count,text
605,"{'screen_name': 'Kelly4220', 'followers_count'...",17540,RT @YoungDems4Trump: Show this to those who do...
837,"{'screen_name': 'GoWithItJam', 'followers_coun...",6426,RT @bfraser747: 💥💥 BREAKING NEWS 💥💥\n\nNobody...
1106,"{'screen_name': 'HijaOfPromise', 'followers_co...",2149,"RT @SenGillibrand: Love this.\n\n""@HillaryClin..."
1100,"{'screen_name': 'winwarchr', 'followers_count'...",2076,"RT @Harlan: Whoa, this is a BRUTAL take on @Hi..."
615,"{'screen_name': 'newstheone', 'followers_count...",1804,RT @RealBenCarson: @HillaryClinton hiding her ...
402,"{'screen_name': 'trolymcmemeface', 'followers_...",1758,RT @DrJillStein: Obama admin including @Hillar...
969,"{'screen_name': 'ofie007', 'followers_count': ...",1587,RT @FoxNews: Fox News Poll: @realDonaldTrump l...
666,"{'screen_name': '3085', 'followers_count': 277...",1586,RT @FoxNews: Fox News Poll: @realDonaldTrump l...
1131,"{'screen_name': 'LifeLessonsApp', 'followers_c...",1392,RT @Jorge_Silva: Celebrating #HispanicHeritage...
465,"{'screen_name': 'I_am_Deplorable', 'followers_...",1385,RT @ImWithYou010: @hillaryclinton talking abou...


We pulled ~1000 of our tweets from the rest API after they'd existed for a period of time, and could be retweeted. These are the 10 most popular.

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary







* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [ ]:
#definitios and config values

api = tweepy.API(auth)

MANGODB_SERVER = 'ds035816.mlab.com'
MANGODB_PORT = 35816
MANGODB_CLINTON_FF_DB_NAME = "clintonff"
MANGODB_CLINTON_FF_COLL = "collection"
db_clinton_ff_client = pymongo.MongoClient(MANGODB_SERVER, MANGODB_PORT)
db_clinton_ff = db_clinton_ff_client[MANGODB_CLINTON_FF_DB_NAME]
db_clinton_ff.authenticate(MANGODB_USER, MANGODB_PASS)  
 

In [ ]:
# fetch all followers 

data_followers = {}
data_followers['type'] = 'followers'
data_followers['lst'] = []
cur = tweepy.Cursor(api.followers_ids, id = 'HillaryClinton').pages()

while True:
    try:
        data_followers['lst'] = cur.next()[:]
        followers_json = json.loads(json.dumps(data_followers))
        db_clinton_ff[MANGODB_CLINTON_FF_COLL].insert_one(followers_json)
        del data_followers['lst'][:]     
    except tweepy.TweepError:
        time.sleep(60 * 15 + 10)
        continue
    except StopIteration:
        break

In [ ]:
# fetch all friends
#no need for iteration like fetching followers as number of friends is relatvely small
data_friends = {}
data_friends['type'] = 'friends'
data_friends['lst'] = []

# fetch all friends list - as this list relativly samll we won't hit the limit here.

friends_ids = []     
for itm in tweepy.Cursor(api.friends_ids, id = 'HillaryClinton').items():
    data_friends['lst'].append(itm)

# save the friend's list in mangoDB    
friends_json = json.loads(json.dumps(data_friends))  
db_clinton_ff[MANGODB_CLINTON_FF_COLL].insert_one(friends_json)

In [ ]:
from prettytable import PrettyTable

# Given a set of user ids, this function calls get_screen_names and plots a table of the first (max_ids) ID's and screen names.
def draw_pretty_table(ids):
    users = api.lookup_users(user_ids=ids)
    pretty_table = PrettyTable(['ID','Screen Name'])
    [pretty_table.add_row ([row.id,row.screen_name]) for row in users]
    pretty_table.align = 'l'
    print(pretty_table)

In [ ]:
# print 20 firends and 20 followers data
#
cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "friends"})
print ("\n20 friends of Hilary Clinton\n")
draw_pretty_table(cursor[0]['lst'][:20])

cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "followers"})
print ("\n20 followers of Hilary Clinton\n")
draw_pretty_table(cursor[0]['lst'][:20])
    

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [11]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "friends"})
friends = cursor[0]['lst']
cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "followers"})

# we have 1 list of friends (~800 members) and over 500 list of followers; each of 5000 members,
#iterate over the followers and build the intersaction list
common_friends = []
total = 0
for follower in cursor:
    common_friends.extend(set(friends).intersection(follower['lst']))
    total += len(follower['lst'])
    
#print ('Clinton has {} followers; {} following ; {} mutal friends'.format(total, len(friends), len(common_friends)))
print("This list is not a complete one as it requires about 30 hours to fetch all followers of \n Clinto due to twitter rate limit\n we fetched about 4M out of 8M followers\n")
print ("Mutual Friends table")
draw_pretty_table(common_friends)

Mutual Friends table
+--------------------+----------------+
| ID                 | Screen Name    |
+--------------------+----------------+
| 3068729259         | HillaryforND   |
| 3065862285         | HillaryforNJ   |
| 172858784          | timkaine       |
| 3064597541         | HillaryforMO   |
| 3044781131         | HillaryforAR   |
| 3073392597         | HillaryforSD   |
| 3078554829         | HillaryforTX   |
| 3057577828         | HillaryforIN   |
| 3073477156         | HillaryforTN   |
| 99545933           | mouto77        |
| 3086798481         | HillaryforWV   |
| 3044746419         | HillaryforCA   |
| 3047899581         | HillaryforCT   |
| 758346398156881920 | HRCforRI       |
| 3060191026         | HillaryforMA   |
| 3057542915         | HillaryforKY   |
| 3048168670         | HillaryforDE   |
| 3060143603         | HillaryforMD   |
| 139939973          | jpbmayor       |
| 4884965834         | HillaryforUT   |
| 755024857373474816 | Hillary_esp    |
| 3044710461       

*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [36]:
# remove this cell before submission - this is only to sync files with DB

lst = []
s = 0
t = 0
db_client = pymongo.MongoClient(MANGODB_SERVER, MANGODB_PORT)
db = db_client[MANGODB_NAME]
db.authenticate(MANGODB_USER, MANGODB_PASS)      
db_collection = db[MANGODB_COLLECTION]
        
with open('data/hillary.json', 'r') as f:
    for line in f:
        try:
            line = line.strip(' \t\n\r')
            tweet = json.loads(line)
            db_collection.insert_one(tweet)
            t = t + 1
        except ValueError:
            s = s + 1 

print("yousef:" + str(s) + " t " + str(t))


yousef:36242 t 5157


In [ ]:
from urllib.parse import urlencode
from urllib.request import Request, urlopen
import json

tweets_Collection = db[MANGODB_COLLECTION]
#tweets_Collection = db['cleaned_tweets']
sentiment_collection = db['sentiment']

cursor = tweets_Collection.find()

for document in cursor:
    
    screen_names = [ user_mention['screen_name']
        for user_mention in document['entities']['user_mentions'] ]
    # exclude tweets that mention both  realDonaldTrump and clinton from sentiment analysis.
    # trying to associate postive/negative in such tweets to clinton or trump is byond this scope of this case.
    if 'realDonaldTrump' in screen_names:
        continue

    # Iterate over the cleaned data - and process sentiment analysis on it. 
    url = 'https://japerk-text-processing.p.mashape.com/sentiment/' # Set destination URL here
    post_fields = {'language': 'english', 'text' : document['text']}     # Set POST fields here

    request = Request(url, urlencode(post_fields).encode())
    # I put my credit card in this website - first 40000 calls are free - DONT EXCEED 
    request.add_header('X-Mashape-Key', '1z19gbsrAwmsh1EqYI3CcBbNtjOSp1T1iVIjsnRoFmszvbDkNV')
    request.add_header('Content-Type', 'application/x-www-form-urlencoded')
    request.add_header('Accept', 'application/json')
    data = urlopen(request).read().decode()
    datajson = json.loads(data)
    sentiment_t = {}
    if document['place']:
        place = document['place']['full_name'].split()
        sentiment_t['state'] = place[(len(place) - 1)]
    sentiment_t['text'] = document['text']
    sentiment_t['sentiment_label'] = datajson['label']
    sentiment_t['entities'] = document['entities']
    sentiment_json = json.loads(json.dumps(sentiment_t))
    sentiment_collection.insert_one(sentiment_json)

print ("done")   

In [ ]:
# example how to get the state from the tweet.
lst = []
states = []
with open('data/hillary.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        lst.append(tweet)

states = [tweeet['place']['full_name'].split()
         for tweeet in lst if tweeet['place']]

states = [x[len(x) - 1] for x in states if x[(len(x) - 1)] != 'USA']      
df = pd.DataFrame(lst)

In [ ]:
import folium
import pandas as pd
from IPython.display import HTML
from base64 import b64encode
%matplotlib inline

state_geo = r'us-states.json'
clinton_popularity_str = '{"WA": 7.8, "DE": 7.1, "WI": 6.8, "WV": 7.5, "HI": 5.4, "FL": 8.2, "WY": 5.1, "NH": 5.7, "NJ": 9.6, "NM": 6.8, "TX": 10, "LA": 5.9, "NC": 9.4, "ND": 3.2, "NE": 3.9, "TN": 7.8, "NY": 8.4, "PA": 8.0, "CA": 10.1, "NV": 10.3, "VA": 5.8, "CO": 7.7, "AK": 6.8, "AL": 7.1, "AR": 7.2, "VT": 5.0, "IL": 8.8, "GA": 8.8, "IN": 8.4, "IA": 5.1, "OK": 5.2, "AZ": 8.1, "ID": 6.6, "CT": 8.4, "ME": 7.2, "MD": 6.8, "MA": 6.7, "OH": 6.9, "UT": 5.5, "MO": 6.7, "MN": 5.6, "MI": 9.1, "RI": 10.1, "KS": 5.6, "MT": 5.8, "MS": 9.1, "SC": 8.8, "KY": 8.1, "OR": 8.5, "SD": 4.4}' 
clinton_popularity_json = json.loads(clinton_popularity_str)

map = folium.Map(location=(39, -100), zoom_start=4)
#map = folium.Map(location=[48, -102], zoom_start=3)
map.choropleth(geo_path=state_geo, data=clinton_popularity_json,
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.8, line_opacity=0.6,
             reset=False)
print ("the map should be shown below; if you can't see it, please find it in DS501map.html or re-run the cell")
map.save("DS501map.html")
HTML(map._repr_html_())

In [ ]:
from prettytable import PrettyTable

def print_top_hashtages(label, tweets, max_to_show):
    
    hashtags = [ hashtag['text'] 
         for tweet in tweets
             for hashtag in tweet['entities']['hashtags'] ]
    pt = PrettyTable(field_names=[label, 'Count'])
    c = Counter(hashtags)
    [ pt.add_row(kv) for kv in c.most_common()[:max_to_show] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)

In [ ]:
# print top hastages in postive and negative tweets

sentiment_collection = db['sentiment']
cursor = sentiment_collection.find({'sentiment_label': 'pos'})
print_top_hashtages('top hashtages in POSTIVE tweets', cursor, 10)

print ("\n\n")
cursor = sentiment_collection.find({'sentiment_label': 'neg'})
print_top_hashtages('top hashtages in NEGATIVE tweets', cursor, 10)


*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

